# Task 1: Predicition
***

Now that we have a model that performs fairly well on all codes, we start filling in out dataset.

In [38]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.util import ngrams

from sklearn.linear_model import SGDClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

>We need our created features class first.

In [39]:
class Feature_Kw(BaseEstimator, TransformerMixin):
    def __init__(self, kw, max_len):
        self.kw = kw
        self.max_len = max_len
        pass

    def fit(self, examples, y=None):
        return self

    def transform(self, examples, y=None):
        
        X = np.zeros((len(examples), len(self.kw)))
        
        tmp = []
        first_pass = True
        while self.max_len > 0:
            for i, abstract in enumerate(examples):
                token = word_tokenize(abstract)
                tmp_grams = list(ngrams(token, self.max_len))
                grams = [' '.join(i) for i in tmp_grams]
                for k in self.kw:
                    if first_pass == True:
                        if k in grams:
                            tmp.append(1)
                        else:
                            tmp.append(0)
                            
                        self.max_len -= 1
                    else:
                        if k in grams:
                            ind = self.kw.index(k)
                            tmp[ind] = 1
                            
                        self.max_len -= 1
                            
                first_pass = False
            
            X[i, :] = np.array(tmp)
        
        return X

In [40]:
pipeline = joblib.load('pipeline.pkl')

>Let us load our dataset for which we want to make predictions for.

In [41]:
df = pd.read_csv('dataframe_pickles/original_frame_1000.csv', encoding='utf-8-sig', low_memory=False)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,70,71,72,73,74,75,76,77,78,79
0,0,Title,Publisher,Abstract,Keywords,JEL Codes,Author1,Author2,Author3,Author4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,technical efficiency of hungarian farms before...,european association of agricultural economists,hungary is one of the ten countries that have ...,"[agribusiness, production economics]",NaN,"zoltán bakucs, lajos","fertő, imre","fogarasi, józsef",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,exploring the flexibility of polish family far...,european association of agricultural economists,no abstract is available for this item,"[farm management, productivity analysis]",NaN,"pieniadz, agata","renner, swetlana","petrick, martin",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,regional clusters in a function of rural devel...,european association of agricultural economists,as a theoretical concept rural development bas...,[community rural urban development],NaN,"sudarić, tihana","zmaić, krunoslav","petrač, božidar",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,the extended metropolitan area in a new member...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"zolin, m. bruna",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,infrastructural capacity of family owned holdi...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, land econo...",NaN,"Živković, dragić","dimitrijević, bojan","jelić, sreten","rajić, zoran",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,comparative research of food consumption in se...,european association of agricultural economists,no abstract is available for this item,[food consumption nutrition food safety],NaN,"Мilanović, milan r.","Đorović, milutin","stevanović, simo",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,targeting agricultural and rural development m...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"segré, andrea","rakić, renata","rokvić, gordana","vittuari, matteo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,needs assessment analysis of small rural house...,european association of agricultural economists,no abstract is available for this item,[consumer household economics],NaN,"bogdanov, natalija","moslavac, nenad",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,designing a rural development strategy for ser...,european association of agricultural economists,no abstract is available for this item,[community rural urban development],NaN,"cochrane, nancy","reed, michael","jovanović, zlatko",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
new_df = df[pd.isnull(df['4'])]
new_df = new_df[['2', '4']]
new_df

,2,4
1,hungary is one of the ten countries that have ...,NaN
2,no abstract is available for this item,NaN
3,as a theoretical concept rural development bas...,NaN
4,no abstract is available for this item,NaN
5,no abstract is available for this item,NaN
6,no abstract is available for this item,NaN
7,no abstract is available for this item,NaN
8,no abstract is available for this item,NaN
9,no abstract is available for this item,NaN
10,no abstract is available for this item,NaN


In [43]:
values_to_predict = new_df['2'].values
values_to_predict

array([ 'hungary is one of the ten countries that have joined the european union eu in may 2004 hungarian farmers are now entitled to receive direct payments per ha in the frame of the single area payment scheme saps while these payments are still lower than the ones received by farmers in the eu 15 they are higher than what hungarian farmers used to receive from national pre accession budget this raises the question of whether accession to the eu has had a positive impact on farmers performance in order to contribute to this issue the paper will investigate technical efficiency of hungarian farmers between 2001 and 2005 using a panel dataset of farms while some studies have investigated other aspects of farm performance in hungary total factor productivity in 1997 by hughes 2000 profitability and total factor productivity in 2000 by davidova et al 2002 there is a clear gap regarding technical efficiency of hungary s farming sector the only post reform paper is by mathijs and vranken 2

In [44]:
prediction = pipeline.predict(values_to_predict)

In [45]:
codes_predict = []
for i in prediction:
    if i == '1':
        codes_predict.append('F')
    else:
        codes_predict.append(np.nan)

In [46]:
new_df['4'] = codes_predict
new_df

,2,4
1,hungary is one of the ten countries that have ...,NaN
2,no abstract is available for this item,NaN
3,as a theoretical concept rural development bas...,NaN
4,no abstract is available for this item,NaN
5,no abstract is available for this item,NaN
6,no abstract is available for this item,NaN
7,no abstract is available for this item,NaN
8,no abstract is available for this item,NaN
9,no abstract is available for this item,NaN
10,no abstract is available for this item,NaN
